This notebook is used to train the k-NN models for key value guesses (from 0 to 255). A total of 256 k-NN models are trained and saved.

Check that the parameters in the dictionary are correct before running this notebook. Verify the train_data_params dictionary under the "Parameters to train 256 k-NN classifiers" heading. The description of each parameter is as follows:

- ~~input_path: The directory containing the dataset for training the k-NN models is specified by input path.~~
- input_path: The file containing the dataset for training the k-NN models is specified by input path.
- target_byte: The attack byte is designated as target byte.
- start_idx: starting index of the attack window.
- end_idx: last index of the attack window.
- testType: The testType parameter is used to select the appropriate dataset with the same or different key as the training dataset. It is always "samekey" when training the k-NN models.
- n: Number of samples to be utilized for training each k-NN model.
- feat_extractor_path: Path to the triplet model to be utilized for extracting the features.
- path_to_save_knns: Path to save the trained k-NN models.

In [1]:
import os, sys
# setting the utilities required for loading the data and training the model
sys.path.append('../utilities/')

from tripletEvalUtilities import *

%load_ext autotime

#### Parameters to train 256 k-NN classifiers

In [2]:
# parameters of the training dataset
train_data_params = {
    "input_path": '../../TripletPower-data/stm32f-unmasked/train-test-256-k-nn/PC2_CB2_TDS3_K6_U_2500/train_same_key.npz', # path load the training data
    "target_byte": 2,  # byte on which attack is to be performed
    "start_idx": 1200,
    "end_idx": 2200,
    "testType": "samekey",  # choices={'samekey', 'diffkey'}
    "n": 2500,
    "feat_extractor_path": '../models/feature-extractor-model/stm32f-unmasked/Triplet_Model_emb_size_512_epochs_10_target_byte_2_nsamples_100.hdf5', # path from where the triplet model is loaded
    "path_to_save_knns": "../models/256-k-nn-models/stm32f-unmasked/" # path to save the trained knn models
}

In [3]:
def gen_features_and_labels_256_2(data, key_byte_value, input_target_byte, start_index, end_index):
    """
    This function generates features and labels for the dataset.
    """

    # loading the dataset
    power_traces, plain_text, key = data['power_trace'], data['plain_text'], data['key']

    print('generating features and labels for the key byte value: ', key_byte_value)

    # key byte is the value between 0 and 15
    labels = []
    for i in range(plain_text.shape[0]):
        text_i = plain_text[i]
        label = aes_internal(text_i[input_target_byte], key_byte_value) #key[i][input_key_byte]
        labels.append(label)

    labels = np.array(labels)
    if not isinstance(power_traces, np.ndarray):
        power_traces = np.array(power_traces)
    power_traces = power_traces[:, start_index:end_index]

    return power_traces, labels

def data_info(data):
    """
    This function prints the information of the dataset.
    """
    # loading the dataset
    power_traces, plain_text, key = data['power_trace'], data['plain_text'], data['key']

    print('shape of the power traces: ', power_traces.shape)
    print('shape of the plaintext : ', plain_text.shape)
    print('Ground Truth for the key : ', key)

In [4]:
def load_data_knn_2(params, key_byte_value, data_label=None):
    """
    This function loads the dataset required.
    """
    print('preparing data for training k-NN model ...')
    target_byte = params['target_byte']
    start_idx, end_idx = params["start_idx"], params["end_idx"]
    file_name = params["input_path"]

    try:
        train_data_whole_pack = np.load(file_name)
    except OSError:
        print("could not access {}".format(file_name))
        sys.exit()
    else:
        data_info(train_data_whole_pack)

    print('-'*80)
    print('processing data for key byte value: ', key_byte_value)
    power_traces, labels = gen_features_and_labels_256_2(train_data_whole_pack,
                                                        key_byte_value,
                                                         target_byte,
                                                        start_idx, end_idx)

    power_traces = power_traces[:params["n"], :]
    labels = labels[:params["n"]]

    print('reshaped power traces: ', power_traces.shape)
    print('shape of the labels: ', labels.shape)

    return power_traces, labels

In [5]:
def train_knn_256(x_train, y_train, key_byte_index=None, model_path=None, neighbours=None, eval_type=None):
    """
    This function trains and test the k-NN model.
    :param x_train: the dataset used for training the k-NN
    :param y_train: the labels corresponding to the power traces in x_train
    :param x_test: the dataset used for evaluating the performance of K-NN
    :param y_test: the labels corresponding to the power traces in x_test
    :param neighbours:  Number of neighbors to use by default for kneighbors queries
    :param eval_type: either N-MEV or N-ALL
    :return: prediction of each testing trace
    """
    # ::TODO: Add data manipulation for N-MEV

    knn = KNeighborsClassifier(n_neighbors=neighbours,
                               # n-shot Number of neighbors to use by default for kneighbors queries. n for n-shot learning
                               weights='distance',
                               p=2,  # Power parameter for the Minkowski metric.
                               metric='cosine',  # the distance metric to use for the tree.
                               algorithm='brute'  # Algorithm used to compute the nearest neighbors
                               )
    knn.fit(x_train, y_train)

    model_name = 'knn-key-value-' + str(key_byte_index) + '.model'
    #     model_name = 'knn-all-class-nshot' + train_data_params["n"] +'.model'
    model_path = os.path.join(model_path, model_name)
    dump(knn, model_path)

    if (key_byte_index % 25) == 0:
        print('model for class %d saved successfully!' % (key_byte_index))
        print('-'*80)

In [6]:
# loading the feature extractor for extracting features of the power traces
input_shape = (train_data_params["end_idx"] - train_data_params["start_idx"], 1)
emb_size = 512
feat_extractor_path = train_data_params["feat_extractor_path"]
feature_extractor_model = load_feature_extractor(feat_extractor_path)
print('-' * 90)

loading the feature extractor for ../models/feature-extractor-model/stm32f-unmasked/Triplet_Model_emb_size_512_epochs_10_target_byte_2_nsamples_100.hdf5 ...
feature extractor loaded successfully!
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1000, 1)]         0         
_________________________________________________________________
block1_conv1 (Conv1D)        (None, 500, 64)           768       
_________________________________________________________________
block1_pool (AveragePooling1 (None, 250, 64)           0         
_________________________________________________________________
block2_conv1 (Conv1D)        (None, 250, 128)          90240     
_________________________________________________________________
block2_pool (AveragePooling1 (None, 125, 128)          0         
________________________________________________________

In [7]:
# training and testing k-NN
eval_type = "N-ALL" # or N-ALL or N-MEV

model_path = os.path.join(train_data_params["path_to_save_knns"], "knn-256-models-n-" + str(train_data_params["n"]) + "-traces-target-byte-" + str(train_data_params["target_byte"]))
if not os.path.isdir(model_path):
    os.makedirs(model_path)

In [8]:
key_byte_values = np.arange(0, 256, 1)

In [9]:
for i in range(len(key_byte_values)):
    print('*'*80)
    
    x_train, y_train = load_data_knn_2(train_data_params, key_byte_values[i], data_label='train')
    # extracting features from the training set and testing set
    x_train_feature_set = extract_features_train(feature_extractor_model, x_train) 
    print('shape of x_train_feature_set: ', x_train_feature_set.shape)
    
    print('Trainng k-NN model ...')
    # train k-NN classifier for respective class
    train_knn_256(x_train_feature_set, y_train, i, model_path=model_path, 
              neighbours=100,
              eval_type=eval_type)
    print('k-NN model for guessed key value %d trained successfully!'%(i))

print('all the 256 k-NN models saved successfully!')

********************************************************************************
preparing data for training k-NN model ...
shape of the power traces:  (20000, 5000)
shape of the plaintext :  (20000, 16)
Ground Truth for the key :  [198  91  97  57 152  71 214 101 138  74  78  84  27 234  87  22]
--------------------------------------------------------------------------------
processing data for key byte value:  0
generating features and labels for the key byte value:  0
reshaped power traces:  (2500, 1000)
shape of the labels:  (2500,)
shape of x_train_feature_set:  (2500, 512)
Trainng k-NN model ...
model for class 0 saved successfully!
--------------------------------------------------------------------------------
k-NN model for guessed key value 0 trained successfully!
********************************************************************************
preparing data for training k-NN model ...
shape of the power traces:  (20000, 5000)
shape of the plaintext :  (20000, 16)
Ground Trut

shape of the power traces:  (20000, 5000)
shape of the plaintext :  (20000, 16)
Ground Truth for the key :  [198  91  97  57 152  71 214 101 138  74  78  84  27 234  87  22]
--------------------------------------------------------------------------------
processing data for key byte value:  12
generating features and labels for the key byte value:  12
reshaped power traces:  (2500, 1000)
shape of the labels:  (2500,)
shape of x_train_feature_set:  (2500, 512)
Trainng k-NN model ...
k-NN model for guessed key value 12 trained successfully!
********************************************************************************
preparing data for training k-NN model ...
shape of the power traces:  (20000, 5000)
shape of the plaintext :  (20000, 16)
Ground Truth for the key :  [198  91  97  57 152  71 214 101 138  74  78  84  27 234  87  22]
--------------------------------------------------------------------------------
processing data for key byte value:  13
generating features and labels for t

generating features and labels for the key byte value:  24
reshaped power traces:  (2500, 1000)
shape of the labels:  (2500,)
shape of x_train_feature_set:  (2500, 512)
Trainng k-NN model ...
k-NN model for guessed key value 24 trained successfully!
********************************************************************************
preparing data for training k-NN model ...
shape of the power traces:  (20000, 5000)
shape of the plaintext :  (20000, 16)
Ground Truth for the key :  [198  91  97  57 152  71 214 101 138  74  78  84  27 234  87  22]
--------------------------------------------------------------------------------
processing data for key byte value:  25
generating features and labels for the key byte value:  25
reshaped power traces:  (2500, 1000)
shape of the labels:  (2500,)
shape of x_train_feature_set:  (2500, 512)
Trainng k-NN model ...
model for class 25 saved successfully!
--------------------------------------------------------------------------------
k-NN model for gues

reshaped power traces:  (2500, 1000)
shape of the labels:  (2500,)
shape of x_train_feature_set:  (2500, 512)
Trainng k-NN model ...
k-NN model for guessed key value 36 trained successfully!
********************************************************************************
preparing data for training k-NN model ...
shape of the power traces:  (20000, 5000)
shape of the plaintext :  (20000, 16)
Ground Truth for the key :  [198  91  97  57 152  71 214 101 138  74  78  84  27 234  87  22]
--------------------------------------------------------------------------------
processing data for key byte value:  37
generating features and labels for the key byte value:  37
reshaped power traces:  (2500, 1000)
shape of the labels:  (2500,)
shape of x_train_feature_set:  (2500, 512)
Trainng k-NN model ...
k-NN model for guessed key value 37 trained successfully!
********************************************************************************
preparing data for training k-NN model ...
shape of the pow

shape of the power traces:  (20000, 5000)
shape of the plaintext :  (20000, 16)
Ground Truth for the key :  [198  91  97  57 152  71 214 101 138  74  78  84  27 234  87  22]
--------------------------------------------------------------------------------
processing data for key byte value:  49
generating features and labels for the key byte value:  49
reshaped power traces:  (2500, 1000)
shape of the labels:  (2500,)
shape of x_train_feature_set:  (2500, 512)
Trainng k-NN model ...
k-NN model for guessed key value 49 trained successfully!
********************************************************************************
preparing data for training k-NN model ...
shape of the power traces:  (20000, 5000)
shape of the plaintext :  (20000, 16)
Ground Truth for the key :  [198  91  97  57 152  71 214 101 138  74  78  84  27 234  87  22]
--------------------------------------------------------------------------------
processing data for key byte value:  50
generating features and labels for t

...

shape of the power traces:  (20000, 5000)
shape of the plaintext :  (20000, 16)
Ground Truth for the key :  [198  91  97  57 152  71 214 101 138  74  78  84  27 234  87  22]
--------------------------------------------------------------------------------
processing data for key byte value:  198
generating features and labels for the key byte value:  198
reshaped power traces:  (2500, 1000)
shape of the labels:  (2500,)
shape of x_train_feature_set:  (2500, 512)
Trainng k-NN model ...
k-NN model for guessed key value 198 trained successfully!
********************************************************************************
preparing data for training k-NN model ...
shape of the power traces:  (20000, 5000)
shape of the plaintext :  (20000, 16)
Ground Truth for the key :  [198  91  97  57 152  71 214 101 138  74  78  84  27 234  87  22]
--------------------------------------------------------------------------------
processing data for key byte value:  199
generating features and labels f

generating features and labels for the key byte value:  210
reshaped power traces:  (2500, 1000)
shape of the labels:  (2500,)
shape of x_train_feature_set:  (2500, 512)
Trainng k-NN model ...
k-NN model for guessed key value 210 trained successfully!
********************************************************************************
preparing data for training k-NN model ...
shape of the power traces:  (20000, 5000)
shape of the plaintext :  (20000, 16)
Ground Truth for the key :  [198  91  97  57 152  71 214 101 138  74  78  84  27 234  87  22]
--------------------------------------------------------------------------------
processing data for key byte value:  211
generating features and labels for the key byte value:  211
reshaped power traces:  (2500, 1000)
shape of the labels:  (2500,)
shape of x_train_feature_set:  (2500, 512)
Trainng k-NN model ...
k-NN model for guessed key value 211 trained successfully!
****************************************************************************

shape of the power traces:  (20000, 5000)
shape of the plaintext :  (20000, 16)
Ground Truth for the key :  [198  91  97  57 152  71 214 101 138  74  78  84  27 234  87  22]
--------------------------------------------------------------------------------
processing data for key byte value:  223
generating features and labels for the key byte value:  223
reshaped power traces:  (2500, 1000)
shape of the labels:  (2500,)
shape of x_train_feature_set:  (2500, 512)
Trainng k-NN model ...
k-NN model for guessed key value 223 trained successfully!
********************************************************************************
preparing data for training k-NN model ...
shape of the power traces:  (20000, 5000)
shape of the plaintext :  (20000, 16)
Ground Truth for the key :  [198  91  97  57 152  71 214 101 138  74  78  84  27 234  87  22]
--------------------------------------------------------------------------------
processing data for key byte value:  224
generating features and labels f

generating features and labels for the key byte value:  235
reshaped power traces:  (2500, 1000)
shape of the labels:  (2500,)
shape of x_train_feature_set:  (2500, 512)
Trainng k-NN model ...
k-NN model for guessed key value 235 trained successfully!
********************************************************************************
preparing data for training k-NN model ...
shape of the power traces:  (20000, 5000)
shape of the plaintext :  (20000, 16)
Ground Truth for the key :  [198  91  97  57 152  71 214 101 138  74  78  84  27 234  87  22]
--------------------------------------------------------------------------------
processing data for key byte value:  236
generating features and labels for the key byte value:  236
reshaped power traces:  (2500, 1000)
shape of the labels:  (2500,)
shape of x_train_feature_set:  (2500, 512)
Trainng k-NN model ...
k-NN model for guessed key value 236 trained successfully!
****************************************************************************

shape of the power traces:  (20000, 5000)
shape of the plaintext :  (20000, 16)
Ground Truth for the key :  [198  91  97  57 152  71 214 101 138  74  78  84  27 234  87  22]
--------------------------------------------------------------------------------
processing data for key byte value:  248
generating features and labels for the key byte value:  248
reshaped power traces:  (2500, 1000)
shape of the labels:  (2500,)
shape of x_train_feature_set:  (2500, 512)
Trainng k-NN model ...
k-NN model for guessed key value 248 trained successfully!
********************************************************************************
preparing data for training k-NN model ...
shape of the power traces:  (20000, 5000)
shape of the plaintext :  (20000, 16)
Ground Truth for the key :  [198  91  97  57 152  71 214 101 138  74  78  84  27 234  87  22]
--------------------------------------------------------------------------------
processing data for key byte value:  249
generating features and labels f